# Create An Adversarial patch for ResNet34

**Install & import required packages**

In [ ]:
pip install torch torchvision numpy matplotlib

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

**import model**

In [ ]:
model = torchvision.models.resnet34(weights='IMAGENET1K_V1')